In [1]:
!pip install -U g4f

In [ ]:
!pip install gdown

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1Dg-1GFfkLrTZz76zPLXHMdMVvOb54VgV

In [2]:
!pip install moviepy

In [ ]:
!pip install pyt2s

In [ ]:
!pip install pydub

In [ ]:
!pip install faster-whisper
!pip install ffmpeg-python

In [3]:
import moviepy.editor as mp
import random

In [4]:
input_video_path = 'template_videos/video0.mov'
output_video_path = 'output_video.mp4'

In [5]:
video = mp.VideoFileClip(input_video_path)

In [6]:
video_duration = video.duration
video_duration

1199.98

# Randomly select a 60-second segment from the video

In [7]:
start_time = random.uniform(0, video_duration - 60)

In [8]:
output_video = video.subclip(start_time, start_time + 60)

# Generate video content

In [9]:
from g4f.client import Client
from g4f import Provider

client = Client()
response = client.chat.completions.create(
    model="command-r+",
    messages=[{"role": "user", "content": "Generate some funny joke for my new video."}],
    provider= Provider.HuggingChat
)

video_text = response.choices[0].message.content
video_text

CohereForAI/c4ai-command-r-plus


"Here are a few jokes that might be suitable for a video:\n\n- Why did the chicken cross the road? To get to the other side!\n- What do you call a bear with no teeth? A gummy bear!\n- Why did the banana go to the doctor? Because he wasn't peeling well!\n- Why did the cookie go to the doctor? Because he was feeling crummy!\n- Why did the tomato turn red? Because it saw the salad dressing!\n- What do you call a pony with a cough? A little horse!\n- Why did the student eat his homework? Because the teacher said it was a piece of cake!\n- What did the shark say when he ate the clown? This tastes a little funny!\n- Why did the ghost go to the doctor? Because he had a bad cold!\n- What do you call a pony with a high IQ? A smarty pants!\n\nThese jokes are lighthearted and playful, and may bring a smile to your viewers' faces."

In [10]:
video_text = video_text.replace('\n-', ' ')
video_text

"Here are a few jokes that might be suitable for a video:\n  Why did the chicken cross the road? To get to the other side!  What do you call a bear with no teeth? A gummy bear!  Why did the banana go to the doctor? Because he wasn't peeling well!  Why did the cookie go to the doctor? Because he was feeling crummy!  Why did the tomato turn red? Because it saw the salad dressing!  What do you call a pony with a cough? A little horse!  Why did the student eat his homework? Because the teacher said it was a piece of cake!  What did the shark say when he ate the clown? This tastes a little funny!  Why did the ghost go to the doctor? Because he had a bad cold!  What do you call a pony with a high IQ? A smarty pants!\n\nThese jokes are lighthearted and playful, and may bring a smile to your viewers' faces."

# Generate audio from content text

In [11]:
from pyt2s.services import stream_elements

data = stream_elements.requestTTS(video_text, stream_elements.Voice.Matthew.value)

In [12]:
from pydub import AudioSegment
from io import BytesIO

def slow_down_audio(audio_bytes, slowdown_percentage):
    slowdown_factor = 1 + slowdown_percentage / 100.0
    
    audio = AudioSegment.from_file(BytesIO(audio_bytes), format="mp3")
    
    new_duration = len(audio) * slowdown_factor
    
    slowed_down_audio = audio._spawn(audio.raw_data, overrides={
        "frame_rate": int(audio.frame_rate / slowdown_factor)
    }).set_frame_rate(audio.frame_rate)
    
    output_io = BytesIO()
    slowed_down_audio.export(output_io, format="wav")
    
    return output_io.getvalue()

# Save the generated audio

In [13]:
video_audio = slow_down_audio(data, 10)

with open('output.mp3', '+wb') as file:
    file.write(video_audio)

In [14]:
import ffmpeg

def extract_audio(input_file):
    extracted_audio = f"audio-{input_file}.wav"
    stream = ffmpeg.input(input_file)
    stream = ffmpeg.output(stream, extracted_audio)
    ffmpeg.run(stream, overwrite_output=True)
    return extracted_audio

In [15]:
audio_extract = extract_audio('output.mp3')

In [16]:
from faster_whisper import WhisperModel
def transcribe(audio):

    model = WhisperModel("small")
    segments, info = model.transcribe(audio, language="en")
    language = info[0]
    print(f" Transcription Language: {language}")
    segments = list(segments) 

    for segment in segments: 
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    return language, segments

In [17]:
language, segments = transcribe(audio_extract)

 Transcription Language: en
[0.00s -> 3.84s]  Here are a few jokes that might be suitable for a video.
[3.84s -> 5.92s]  Why did the chicken cross the road?
[5.92s -> 7.68s]  To get to the other side.
[7.68s -> 9.76s]  What do you call a bear with no teeth?
[9.76s -> 11.16s]  A gummy bear.
[11.16s -> 13.64s]  Why did the banana go to the doctor?
[13.64s -> 15.60s]  Because he wasn't feeling well.
[15.60s -> 18.00s]  Why did the cookie go to the doctor?
[18.00s -> 19.88s]  Because he was feeling crummy.
[19.88s -> 22.20s]  Why did the tomato turn red?
[22.20s -> 24.32s]  Because it saw the salad dressing.
[24.32s -> 26.48s]  What do you call a pony with a cough?
[26.48s -> 27.96s]  A little horse.
[27.96s -> 30.44s]  Why did the student eat his homework?
[30.44s -> 33.04s]  Because the teacher said it was a piece of cake.
[33.04s -> 35.76s]  What did the shark say when he ate the clown?
[35.76s -> 37.64s]  This tastes a little funny.
[37.64s -> 40.08s]  Why did the ghost go to the docto

In [18]:
import math 
def format_time_for_srt(seconds):
    hours = math.floor(seconds / 3600)
    seconds %= 3600
    minutes = math.floor(seconds / 60)
    seconds %=60
    milliseconds = round((seconds - math.floor(seconds)) * 1000)
    seconds = math.floor(seconds)
    formatted_time = f"{hours :02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

    return formatted_time

In [19]:
def generate_subtitle_file(input_file, language, segments):
    subtitle_file = f"sub-{input_file}.{language}.srt"
    text = ""
    for index, segment in enumerate(segments):
        segment_start = format_time_for_srt(segment.start)
        segment_end = format_time_for_srt(segment.end)

        text += f"{str(index + 1)}\n"
        text += f"{segment_start} --> {segment_end}\n"
        text += f"{segment.text}\n\n"
    
    f = open(subtitle_file, "w")
    f.write(text)
    f.close()

    return subtitle_file

In [20]:
subtitle_file = generate_subtitle_file('output', language, segments)

In [21]:
%env IMAGEMAGICK_BINARY="C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"
import os
os.environ['IMAGEMAGICK_BINARY'] = "C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"
from moviepy.config import change_settings

change_settings({"IMAGEMAGICK_BINARY": "C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})


env: IMAGEMAGICK_BINARY="C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"


# Save the output video

In [24]:
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.video.VideoClip import TextClip
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip

generator = lambda txt: TextClip(txt, font='./fonts/Super_Caramel.ttf', fontsize=96, color='white', stroke_color="black", stroke_width=2, method='caption', align='center', size=output_video.size)
subtitles = SubtitlesClip(subtitle_file, generator)
audio = AudioFileClip('output.mp3')
audio_duration = audio.duration

audio_duration

50.69

In [25]:

output_video = output_video.set_duration(audio_duration)

output_video = output_video.set_audio(audio)
results = CompositeVideoClip([output_video, subtitles.set_pos(('center', 'center'))])

results.write_videofile(output_video_path, codec='libx264')

Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4


Moviepy - Done !
Moviepy - video ready output_video.mp4
